### Install project

In [1]:
!git clone https://github.com/kornelro/cv_project.git

fatal: destination path 'cv_project' already exists and is not an empty directory.


In [2]:
!pip install cv_project/.

Processing ./cv_project
  Created wheel for src: filename=src-0.1.0-cp36-none-any.whl size=16556 sha256=bef86d8c3b393edcae373f9b3adec9542d30d9a84d0a70bcfcc98af443d3d65e
  Stored in directory: /tmp/pip-ephem-wheel-cache-mjwp3vh9/wheels/fc/1b/40/72379bece2bafba26136079aeac87421c9221abafeb9c66746
Successfully built src
  Found existing installation: src 0.1.0
    Uninstalling src-0.1.0:
      Successfully uninstalled src-0.1.0


### Mount drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
DRIVE_PATH = '/content/drive/Shareddrives/CV-PROJECT/'

### Imports

In [5]:
import pickle
import numpy as np
import tensorflow.keras as K
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from src.data.make_dataset import make_dataset
from src.data.image_loaders import OrtoFixedSizeImageLoader
from src.data.dataset_loaders import OrtoDatasetLoader
from src.features.pipelines import RawImageToImage
from src.features.processors import NormImage
from src.models.classifiers import ResnetModelWrapper
from src.models.detectors import SliderProbDetector
from src.data.sliders import SlidingWindow
from src.evaluation import validate_model, precision, recall, f1_score

# Orto
# ResNet50
# Pipeline: Norm

### Load dataset

In [6]:
DATA_PATH = DRIVE_PATH+'data/processed/aerial-cars-dataset/resnet/data_resnet.pickle'

In [7]:
with open(DATA_PATH, 'rb') as f:
    data = pickle.load(f)

### Train ResNet50

In [8]:
TEST_SIZE = 0.1
BATCH_SIZE = 100
EPOCHS = 5

In [9]:
X = np.array(list(map(lambda x: x[1], data)))
y = np.array(list(map(lambda x: x[2], data)))
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=TEST_SIZE,
    random_state=42
  )


input_t = K.Input(shape=(X[0].shape))
res_model = K.applications.ResNet50V2(
    include_top=False,
    weights='imagenet',
    input_tensor=input_t
)

for layer in res_model.layers:
    layer.trainable = False

model = K.models.Sequential()
model.add(res_model)
model.add(K.layers.Flatten())
model.add(K.layers.Dense(2, activation='softmax'))


model.compile(
    optimizer=K.optimizers.Adam(0.001),
    loss=K.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

history = model.fit(X_train, y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS, 
            validation_data=(X_test, y_test)
)

Epoch 1/5
43/43 [==============================] - 9s 89ms/step - loss: 0.2234 - accuracy: 0.9403 - val_loss: 0.0897 - val_accuracy: 0.9852
Epoch 2/5
43/43 [==============================] - 2s 47ms/step - loss: 0.0231 - accuracy: 0.9942 - val_loss: 0.1517 - val_accuracy: 0.9788
Epoch 3/5
43/43 [==============================] - 2s 47ms/step - loss: 0.0100 - accuracy: 0.9984 - val_loss: 0.0812 - val_accuracy: 0.9873
Epoch 4/5
43/43 [==============================] - 2s 47ms/step - loss: 8.4882e-04 - accuracy: 1.0000 - val_loss: 0.0797 - val_accuracy: 0.9894
Epoch 5/5
43/43 [==============================] - 2s 47ms/step - loss: 2.9335e-04 - accuracy: 1.0000 - val_loss: 0.0847 - val_accuracy: 0.9873


### Validation

In [10]:
del data

In [11]:
BND_BOX_SIZE = (80, 80)
STEP_SIZE = 20

dataset_loader = OrtoDatasetLoader(
    image_loader=OrtoFixedSizeImageLoader(
        bnd_box_size=BND_BOX_SIZE
    )
)

process_pipeline=RawImageToImage(
    processors=[
        NormImage()
    ]
)

sliding_window = SlidingWindow(
    step_size=STEP_SIZE,
    window_size=BND_BOX_SIZE
)

treshold = 0.9; nms_overlap=0.4

In [18]:
slider_detector = SliderProbDetector(
    sliding_window=sliding_window,
    process_pipeline=process_pipeline,
    classifier=ResnetModelWrapper(model),
    treshold=0.9,
    nms_overlap=0.4
)

true_positives, false_positives, false_negatives, processed_images = validate_model(
    dataset_loader=dataset_loader,
    input_folder_filepath=DRIVE_PATH+'data/raw/orto/val',
    output_folder_filepath=DRIVE_PATH+'processed_images/resnet50/orto/norm/t9nms4',
    images_files_types=('png',),
    annotations_files_types=('json',),
    detector=slider_detector,
    workers=0
)


print()
print('Precision:',  round(precision(true_positives, false_positives), 3))
print('Recall:',  round(recall(true_positives, false_negatives), 3))
print('F1Score:', round(f1_score(true_positives, false_positives, false_negatives), 3))

  0%|          | 0/5 [00:00<?, ?it/s]

538


 40%|████      | 2/5 [00:26<00:39, 13.14s/it]

122
397


 60%|██████    | 3/5 [00:39<00:26, 13.14s/it]

359


 80%|████████  | 4/5 [00:52<00:13, 13.16s/it]

590


100%|██████████| 5/5 [01:05<00:00, 13.17s/it]



Precision: 0.43
Recall: 0.622
F1Score: 0.508


treshold = 0.95; nms_overlap=0.4

In [19]:
del processed_images

In [20]:
slider_detector = SliderProbDetector(
    sliding_window=sliding_window,
    process_pipeline=process_pipeline,
    classifier=ResnetModelWrapper(model),
    treshold=0.95,
    nms_overlap=0.4
)

true_positives, false_positives, false_negatives, processed_images = validate_model(
    dataset_loader=dataset_loader,
    input_folder_filepath=DRIVE_PATH+'data/raw/orto/val',
    output_folder_filepath=DRIVE_PATH+'processed_images/resnet50/orto/norm/t95nms4',
    images_files_types=('png',),
    annotations_files_types=('json',),
    detector=slider_detector,
    workers=0
)


print()
print('Precision:',  round(precision(true_positives, false_positives), 3))
print('Recall:',  round(recall(true_positives, false_negatives), 3))
print('F1Score:', round(f1_score(true_positives, false_positives, false_negatives), 3))

  0%|          | 0/5 [00:00<?, ?it/s]

500


 40%|████      | 2/5 [00:26<00:39, 13.14s/it]

116
371


 60%|██████    | 3/5 [00:39<00:26, 13.23s/it]

339


 80%|████████  | 4/5 [00:52<00:13, 13.20s/it]

558


100%|██████████| 5/5 [01:06<00:00, 13.22s/it]



Precision: 0.447
Recall: 0.604
F1Score: 0.514


treshold = 0.98; nms_overlap=0.4

In [22]:
del processed_images

In [23]:
slider_detector = SliderProbDetector(
    sliding_window=sliding_window,
    process_pipeline=process_pipeline,
    classifier=ResnetModelWrapper(model),
    treshold=0.9,
    nms_overlap=0.3
)

true_positives, false_positives, false_negatives, processed_images = validate_model(
    dataset_loader=dataset_loader,
    input_folder_filepath=DRIVE_PATH+'data/raw/orto/val',
    output_folder_filepath=DRIVE_PATH+'processed_images/resnet50/orto/norm/t98nms4',
    images_files_types=('png',),
    annotations_files_types=('json',),
    detector=slider_detector,
    workers=0
)


print()
print('Precision:',  round(precision(true_positives, false_positives), 3))
print('Recall:',  round(recall(true_positives, false_negatives), 3))
print('F1Score:', round(f1_score(true_positives, false_positives, false_negatives), 3))

  0%|          | 0/5 [00:00<?, ?it/s]

538


 40%|████      | 2/5 [00:26<00:39, 13.15s/it]

122
397


 80%|████████  | 4/5 [00:52<00:13, 13.15s/it]

359
590


100%|██████████| 5/5 [01:05<00:00, 13.15s/it]



Precision: 0.471
Recall: 0.518
F1Score: 0.493


treshold = 0.95; nms_overlap=0.5

In [ ]:
del processed_images

In [24]:
slider_detector = SliderProbDetector(
    sliding_window=sliding_window,
    process_pipeline=process_pipeline,
    classifier=ResnetModelWrapper(model),
    treshold=0.95,
    nms_overlap=0.5
)

true_positives, false_positives, false_negatives, processed_images = validate_model(
    dataset_loader=dataset_loader,
    input_folder_filepath=DRIVE_PATH+'data/raw/orto/val',
    output_folder_filepath=DRIVE_PATH+'processed_images/resnet50/orto/norm/t95nms5',
    images_files_types=('png',),
    annotations_files_types=('json',),
    detector=slider_detector,
    workers=0
)


print()
print('Precision:',  round(precision(true_positives, false_positives), 3))
print('Recall:',  round(recall(true_positives, false_negatives), 3))
print('F1Score:', round(f1_score(true_positives, false_positives, false_negatives), 3))

  0%|          | 0/5 [00:00<?, ?it/s]

500


 40%|████      | 2/5 [00:26<00:39, 13.12s/it]

116
371


 60%|██████    | 3/5 [00:39<00:26, 13.15s/it]

339


 80%|████████  | 4/5 [00:53<00:13, 13.26s/it]

558


100%|██████████| 5/5 [01:06<00:00, 13.25s/it]



Precision: 0.447
Recall: 0.604
F1Score: 0.514


treshold = 0.95; nms_overlap=0.3

In [25]:
del processed_images

In [26]:
slider_detector = SliderProbDetector(
    sliding_window=sliding_window,
    process_pipeline=process_pipeline,
    classifier=ResnetModelWrapper(model),
    treshold=0.95,
    nms_overlap=0.3
)

true_positives, false_positives, false_negatives, processed_images = validate_model(
    dataset_loader=dataset_loader,
    input_folder_filepath=DRIVE_PATH+'data/raw/orto/val',
    output_folder_filepath=DRIVE_PATH+'processed_images/resnet50/orto/norm/t95nms3',
    images_files_types=('png',),
    annotations_files_types=('json',),
    detector=slider_detector,
    workers=0
)


print()
print('Precision:',  round(precision(true_positives, false_positives), 3))
print('Recall:',  round(recall(true_positives, false_negatives), 3))
print('F1Score:', round(f1_score(true_positives, false_positives, false_negatives), 3))

  0%|          | 0/5 [00:00<?, ?it/s]

500


 40%|████      | 2/5 [00:26<00:39, 13.10s/it]

116


 60%|██████    | 3/5 [00:39<00:26, 13.17s/it]

371


 80%|████████  | 4/5 [00:52<00:13, 13.15s/it]

339
558


100%|██████████| 5/5 [01:05<00:00, 13.17s/it]



Precision: 0.505
Recall: 0.526
F1Score: 0.515
